In [1]:
import numpy as np
import re
import random

import tensorflow as tf

import Functions as fn
from DS import DS
from Set import pool
from Iterator import Iterator

Loading BokehJS ...

In [2]:
#Dataset = fn.first_time_load()
#Dataset.writeTexts()
#Dataset.writeLabels()

In [3]:
Dataset = pool()
Dataset.loadTexts()
Dataset.loadLabels()

Texts loaded
Labels loaded


In [4]:
print('Number of Texts: ', Dataset.size)
print('Number of 2007 Smoking Challenge texts: ', Dataset.numberOf(challenge='2007 Smoking Challenge'))
print('Number of 2008 ObesityChallenge texts: ', Dataset.numberOf(challenge='2008 Obesity Challenge'))
print('Number of 2009 Medication Challenge texts: ', Dataset.numberOf(challenge='2009 Medication Challenge'))
print('Number of 2010 Relations Challenge texts: ', Dataset.numberOf(challenge='2010 Relations Challenge'))
print('Number of 2011 Coreference Challenge texts: ', Dataset.numberOf(challenge='2011 Coreference Challenge'))
print('Number of 2012 Temporal Relations Challenge texts: ', Dataset.numberOf(challenge='2012 Temporal Relations Challenge'))
print('Number of Train Texts: ', Dataset.numberOf(stage='train'))
print('Number of Test Texts: ', Dataset.numberOf(stage='test'))
print('Number of Labeled Texts: ', Dataset.numberOf(labelled='yes'))
print('Number of Initially Labeled Texts: ', Dataset.numberOf(labelled='yes', label_type='train'))
print('Number of Competitor Labeled Texts Texts: ', Dataset.numberOf(labelled='yes', label_type='test'))

Number of Texts:  4605
Number of 2007 Smoking Challenge texts:  926
Number of 2008 ObesityChallenge texts:  1237
Number of 2009 Medication Challenge texts:  1249
Number of 2010 Relations Challenge texts:  694
Number of 2011 Coreference Challenge texts:  188
Number of 2012 Temporal Relations Challenge texts:  311
Number of Train Texts:  2859
Number of Test Texts:  1746
Number of Labeled Texts:  258
Number of Initially Labeled Texts:  10
Number of Competitor Labeled Texts Texts:  248


In [5]:
Dataset.processForEmbedding()
sentences = Dataset.getSentences()

In [6]:
medications, dosages, modes, frequencies, durations, reasons = fn.label_words(Dataset)

Number of: m=928, do=119, mo=76, f=79, du=80, r=482


In [7]:
from gensim.models import Word2Vec

C:\Users\Luka\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [8]:
model_I2B2 = Word2Vec(sentences, min_count=1, size=100)

In [9]:
#model_I2B2.most_similar("doctor")

In [10]:
fn.visualise(model=model_I2B2, sentences=sentences, labels=[medications, reasons], topn=1000)

In [ ]:
train_set = []
train_labels = []
train_size = 2000
vocab = set(model_I2B2.wv.vocab.keys())
target = set(medications)

for i in range(train_size * 9 // 10):
    word = random.sample(vocab, 1)
    train_set.append(model_I2B2[word[0]])
    vocab.discard(word[0])
    if word[0] in target:
        train_labels.append([1,0])
        target.discard(word[0])
    else:
        train_labels.append([0,1])
    if i % 10 == 0:
        word = random.sample(target, 1)
        train_set.append(model_I2B2[word[0]])
        train_labels.append([1,0])
        target.discard(word[0])

In [ ]:
np.array(train_labels).sum(0)/len(train_labels)

In [ ]:
test_set = []
test_labels = []
test_size = 500

for i in range(int(test_size * 0.5)):
    word = random.sample(vocab, 1)
    test_set.append(model_I2B2[word[0]])
    vocab.discard(word[0])
    if word[0] in medications:
        test_labels.append([1,0])
        target.discard(word[0])
    else:
        test_labels.append([0,1])
        
for i in range(int(test_size * 0.5)):
    word = random.sample(target, 1)
    test_set.append(model_I2B2[word[0]])
    test_labels.append([1,0])
    target.discard(word[0])

In [ ]:
np.array(test_labels).sum(0)/len(test_labels)

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.05)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
node_count_1 = 50

x = tf.placeholder(tf.float32, shape=[None, 100])
y_ = tf.placeholder(tf.float32, shape=[None, 2])

# Define the first layer here
W = weight_variable([100, node_count_1])
b = bias_variable([node_count_1])
h = tf.nn.sigmoid(tf.matmul(x, W) + b)

# Use dropout for this layer (should you wish)
#keep_prob = tf.placeholder(tf.float32)
#h_drop = tf.nn.dropout(h1, keep_prob)

# Define the output layer here
V = weight_variable([node_count_1, 2])
c = bias_variable([2])
y = tf.nn.softmax(tf.matmul(h, V) + c)

# We'll use the cross entropy loss function 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))

# And classification accuracy
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# And the Adam optimiser
train_step = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(cross_entropy)

# Start a tf session and run the optimisation algorithm
sess = tf.Session()
sess.run(tf.initialize_all_variables())

training = Iterator(train_set, train_labels)
train_accuracy = 0
test_accuracy = 0
N = 0

while training.epochs < 1000:
    trd, trl = training.next_batch(50)
    if N % 4000 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x: trd, y_: trl})
        test_accuracy = sess.run(accuracy, feed_dict={x: test_set, y_: test_labels})
        print("Epoch: %d, Training Accuracy: %f, Test Accuracy: %f" % (training.epochs, train_accuracy, test_accuracy))
    sess.run(train_step, feed_dict={x: trd, y_: trl})
    N += 1